<a href="https://colab.research.google.com/github/conniaren/GenotypeImputationProject/blob/master/2-Models/autoencoder_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install wandb -qqq

In [7]:
pip install pytorch-lightning

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import scipy
from scipy import sparse 
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchvision import transforms
from torchvision.transforms import Compose
from torch import nn, optim 
import wandb
import pytorch_lightning as pl
import torch.nn.functional as F

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
wandb.login()

wandb: Currently logged in as: connia (use `wandb login --relogin` to force relogin)


True

In [3]:
#upload dataset
dataset = sparse.load_npz('gdrive/My Drive/Colab Notebooks/output_matrix.npz')
dense_dataset = dataset.todense()
genotype_dataset = dense_dataset

'''
#adding 5% noise to the data (setting random genotypes to 0)
for i in range(dense_dataset.shape[0]):
  noise_percent = int(0.05 * dense_dataset.shape[1])
  noise_indices = np.random.randint(dense_dataset.shape[1], size= noise_percent)
  dense_dataset[i, noise_indices] = 0
'''

class NoisyDataset(Dataset):
  
  def __init__(self,datasetnoised,datasetclean,transform):
    self.noise=datasetnoised
    self.clean=datasetclean
    self.transform=transform
  
  def __len__(self):
    return len(self.noise)
  
  def __getitem__(self,idx):
    xNoise=self.noise[idx]
    xClean=self.clean[idx]
    
    if self.transform != None:
      xNoise=self.transform(xNoise)
      xClean=self.transform(xClean)
    return (xNoise,xClean)

#initialize dataloader for dataset

transform=Compose([
    transforms.ToTensor()
])

#genotype_dataset=NoisyDataset(dense_dataset,clean,transform)
#print (genotype_dataset[0])
genotype_dataset = TensorDataset(torch.tensor(genotype_dataset, dtype=torch.float64))
dataloader = DataLoader(genotype_dataset, batch_size = 10, shuffle = True, num_workers=1)

class autoencoder_model_1 (pl.LightningModule):
  def __init__(self, **kwargs):
    super().__init__()
    self.encoder = nn.Sequential(nn.Linear(kwargs["input_shape"],256), nn.ReLU(), nn.Linear(256,256))
    self.decoder = nn.Sequential(nn.Linear(256,256), nn.ReLU(), nn.Linear(256,kwargs["input_shape"]))
    self.double()

  def forward (self, features):
    reconstruction = self.encoder(features)
    return reconstruction
  
  def training_step(self, batch, batch_idx):
        # training_step defined the train loop.
        # It is independent of forward
        x = batch
        x = x[0].view(x[0].size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = F.mse_loss(x_hat, x)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        wandb.log({ "loss": loss})
        return loss
  
  def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=1e-3)
        return optimizer

In [4]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = autoencoder_model_1(input_shape = 264695)
#optimizer = optim.Adam(model.parameters(), lr = 1e-3)
#reconstruction_loss = nn.MSELoss()
trainer = pl.Trainer()



GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1295: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer"


In [6]:
args = {
      "learning_rate": 0.001,
      "architecture": "Autoencoder",
      "dataset": "1000 Genome Project",
      "layer_1": 256,
      "activation_1": "relu",
      "layer_2": 256,
      "activation_2": "relu",
      "optimizer": "Adam",
      "loss": "MSE",
      "metric": "accuracy",
      "epoch": 25,
      "batch_size": 1}
group_name = "hidden_layer_size_group"
name=group_name+'_seed_'+str(np.random.randint(100000000))
run=wandb.init(project="Imputation Autoencoder Project",save_code=False,
               group=group_name,entity="connia",name=name,
               mode="online",id=name,config=args,allow_val_change=True)

In [ ]:
trainer.fit(model, dataloader)


  | Name    | Type       | Params
---------------------------------------
0 | encoder | Sequential | 67.8 M
1 | decoder | Sequential | 68.1 M
---------------------------------------
135 M     Trainable params
0         Non-trainable params
135 M     Total params
543.682   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [37]:
epochs = 25
for epoch in range(epochs):
  loss = 0 
  for batch_features in dataloader:
    #unpack 
    batch_features = batch_features[0].view(-1,264695).type(torch.DoubleTensor).to(device)
    #clean_features = clean_features.view(264695,-1).type(torch.DoubleTensor).to(device)
    #reset gradients to 0
    optimizer.zero_grad()
    # forward propogation 
    outputs = model(batch_features)
    # calculate the loss
    train_loss = reconstruction_loss(outputs,batch_features)
    #backprop
    train_loss.backward()
    #update parameters with Adam optimizer 
    optimizer.step()
    # sum the total loss across all samples
    loss += train_loss.item()

  # take the average loss in an epoch
  loss = loss/len(dataloader)
  print( "epoch: {}/{}, loss: {:.6f}".format(epoch+1, epochs, loss))
  wandb.log({ "loss": loss})

epoch: 1/25, loss: 0.449878
epoch: 2/25, loss: 0.311405
epoch: 3/25, loss: 0.304939
epoch: 4/25, loss: 0.268539
epoch: 5/25, loss: 0.267630
epoch: 6/25, loss: 0.266968
epoch: 7/25, loss: 0.266631
epoch: 8/25, loss: 0.266566
epoch: 9/25, loss: 0.267111
epoch: 10/25, loss: 0.266164
epoch: 11/25, loss: 0.265816
epoch: 12/25, loss: 0.266767
epoch: 13/25, loss: 0.265539
epoch: 14/25, loss: 0.264806
epoch: 15/25, loss: 0.264682
epoch: 16/25, loss: 0.264629
epoch: 17/25, loss: 0.264544
epoch: 18/25, loss: 0.264626
epoch: 19/25, loss: 0.264762
epoch: 20/25, loss: 0.264847
epoch: 21/25, loss: 0.265092
epoch: 22/25, loss: 0.265423
epoch: 23/25, loss: 0.265291
epoch: 24/25, loss: 0.265123
epoch: 25/25, loss: 0.264896


In [1]:
 wandb.finish()

NameError: ignored

In [6]:
print (model)

autoencoder_model_1(
  (input_layer): Linear(in_features=264695, out_features=256, bias=True)
  (output_layer): Linear(in_features=256, out_features=264695, bias=True)
)


In [ ]:
torch.save(model.state_dict(), 'checkpoint.pth')
from google.colab import files 
files.download('checkpoint.pth')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
state_dict = torch.load('checkpoint.pth')
model.load_state_dict(state_dict)
print(model)

autoencoder_model_1(
  (input_layer): Linear(in_features=264695, out_features=64, bias=True)
  (encoder_layer): Linear(in_features=64, out_features=64, bias=True)
  (decoder_layer): Linear(in_features=64, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=264695, bias=True)
)
